# Summary of notebook

In this notebook, we perform the following analysis:
(1) Clustering of the cells of young and old mice, annotation of the clusters using marker genes of immune cells and adipocytes. 
(2) Separate the preadipocytes from the immune cells and identify the mature subpopulation.

# Load packages and set global variables

In [1]:
import numpy as np
import scanpy as sc
import scipy as sci
import scipy.sparse
import gseapy
import re
import pandas
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sys

from matplotlib import colors

import batchglm
import diffxpy.api as de

#from beakerx import *

%load_ext autoreload
%autoreload 2

sc.settings.verbosity = 3 # amount of output
dir_in = '/Users/viktorian.miok/Documents/consultation/Altun-Ussar/igf2/data/'
dir_out = '/Users/viktorian.miok/Documents/consultation/Altun-Ussar/igf2/results/'
dir_mat = '/count_matrices/filtered_gene_bc_matrices/mm10_ensrel94/'
dir_tables = dir_out+'tables/'
sc_settings_figdir = dir_out+'panels/'
sc_settings_writedir = dir_out+'anndata/'
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80, scanpy=True)
print (sys.version)

-----
anndata     0.7.5
scanpy      1.7.1
sinfo       0.3.1
-----
PIL                 8.1.2
PyObjCTools         NA
anndata             0.7.5
appdirs             1.4.4
appnope             0.1.2
autoreload          NA
backcall            0.2.0
batchglm            v0.7.4
bioservices         1.7.11
bs4                 4.9.3
certifi             2020.12.05
cffi                1.14.5
chardet             4.0.0
cloudpickle         1.6.0
colorama            0.4.4
colorlog            NA
cycler              0.10.0
cython_runtime      NA
dask                2021.03.0
dateutil            2.8.1
decorator           4.4.2
diffxpy             v0.7.4
docutils            0.16
easydev             0.11.0
get_version         2.1
gseapy              0.10.4
h5py                3.2.1
idna                2.10
igraph              0.9.0
ipykernel           5.4.3
ipython_genutils    0.2.0
ipywidgets          7.6.3
jedi                0.17.2
joblib              1.0.1
kiwisolver          1.3.1
legacy_api_wrap     1.2

In [2]:
print(de.__version__)

v0.7.4


## Global variables

All embeddings and clusterings can be saved and loaded into this script. Be careful with over-writing cluster caches as soon as cell type annotation has started as cluster labels may be shuffled.

Set whether anndata objects are recomputed or loaded from cache.

In [3]:
bool_recomp = False

Set whether clustering is recomputed or loaded from saved .obs file. Loading makes sense if the clustering changes due to a change in scanpy or one of its dependencies and the number of clusters or the cluster labels change accordingly.

In [4]:
bool_recluster = False

Set whether cluster cache is overwritten. Note that the cache exists for reproducibility of clustering, see above.

In [5]:
bool_write_cluster_cache = False

Set whether to produce plots, set to False for test runs.

In [6]:
bool_plot = False

# Load data

In [7]:
if bool_recomp:
    # Count matrix:
    adata87 = sc.read_10x_mtx(dir_in+'MUC8387'+dir_mat, var_names='gene_symbols', cache=False)  
    adata88 = sc.read_10x_mtx(dir_in+'MUC8388'+dir_mat, var_names='gene_symbols', cache=False)  
    adata89 = sc.read_10x_mtx(dir_in+'MUC8389'+dir_mat, var_names='gene_symbols', cache=False)  
    adata90 = sc.read_10x_mtx(dir_in+'MUC8390'+dir_mat, var_names='gene_symbols', cache=False)  
    adataY = sc.read_10x_mtx(dir_in+'PreYoung'+dir_mat, var_names='gene_symbols', cache=False)  
    adataO = sc.read_10x_mtx(dir_in+'PreOld'+dir_mat, var_names='gene_symbols', cache=False)  
    adata_raw = adata87.concatenate(adata88, adata89, adata90, adataY, adataO)
    p = adata_raw.obs_names.str.endswith
    groups = ['BAT-2','BAT-8','PGF-2','PGF-8','SCF-2','SCF-8']
    adata_raw.obs['batch_type'] = np.select([p('-0'),p('-1'),p('-2'),p('-3'),p('-4'),p('-5')], groups)
    adata_raw.obs['tissue_type'] = np.select([(p('-0')|p('-1')),(p('-2')|p('-3')),(p('-4')|p('-5'))], ['BAT', 'PGF','SCF'])
    adata_raw.obs['age_type'] = np.select([(p('-0')|p('-2')|p('-4')),(p('-1')|p('-3')|p('-5'))], ['young', 'old'])
    sc.write(sc_settings_writedir+'adata_raw.h5ad', adata_raw)
else:
    adata_raw = sc.read(sc_settings_writedir+'adata_raw.h5ad')

# Filtering

In [8]:
adata_raw.obs['n_counts'] = adata_raw.X.sum(1)
adata_raw.obs['log_counts'] = np.log(adata_raw.obs['n_counts'])
adata_raw.obs['n_genes'] = (adata_raw.X > 0).sum(1)

In [9]:
mt_gene_mask = np.flatnonzero([gene.startswith('mt-') for gene in adata_raw.var_names])
adata_raw.obs['mt_frac'] =adata_raw.X[:, mt_gene_mask].todense().sum(1).flat/adata_raw.obs['n_counts']

In [10]:
if bool_plot==True:
    t1 = sns.distplot(adata_raw.obs['mt_frac'][adata_raw.obs['mt_frac']<0.12], kde=False)

In [11]:
if bool_plot==True:
    p3 = sns.distplot(adata_raw.obs['n_counts'], kde=False)
    plt.show()

    p4 = sns.distplot(adata_raw.obs['n_counts'][adata_raw.obs['n_counts']<4000], kde=False, bins=60)
    plt.show()

    p5 = sns.distplot(adata_raw.obs['n_counts'][adata_raw.obs['n_counts']>20000], kde=False, bins=60)
    plt.show()

In [12]:
if bool_plot==True:
    p6 = sns.distplot(adata_raw.obs['n_genes'], kde=False, bins=60)
    plt.show()

    p7 = sns.distplot(adata_raw.obs['n_genes'][adata_raw.obs['n_genes']<800], kde=False, bins=60)
    plt.show()

In [13]:
#Filter cells according to identified QC thresholds:
print('Total number of cells: {:d}'.format(adata_raw.n_obs))

sc.pp.filter_cells(adata_raw, min_counts = 2000)
print('Number of cells after min count filter: {:d}'.format(adata_raw.n_obs))

sc.pp.filter_cells(adata_raw, max_counts = 25000)
print('Number of cells after max count filter: {:d}'.format(adata_raw.n_obs))

adata = adata_raw[adata_raw.obs['mt_frac'] < 0.10]
print('Number of cells after MT filter: {:d}'.format(adata_raw.n_obs))

sc.pp.filter_cells(adata_raw, min_genes = 800)
print('Number of cells after gene filter: {:d}'.format(adata_raw.n_obs))

filtered out 2610 cells that have less than 2000 counts


Total number of cells: 24733


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
filtered out 66 cells that have more than 25000 counts


Number of cells after min count filter: 22123
Number of cells after max count filter: 22057
Number of cells after MT filter: 22057


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
filtered out 1318 cells that have less than 800 genes expressed


Number of cells after gene filter: 20739


In [14]:
#Filter genes:
print('Total number of genes: {:d}'.format(adata_raw.n_vars))

# Min 20 cells - filters out 0 count genes
sc.pp.filter_genes(adata_raw, min_cells=20)
print('Number of genes after cell filter: {:d}'.format(adata_raw.n_vars))

Total number of genes: 31125


filtered out 13331 genes that are detected in less than 20 cells


Number of genes after cell filter: 17794


# Process data

## Embeddings and clustering

Summary of steps performed here: Only cells with at least 500 UMIs are kept. Counts per cell are cell library depth normalized. The gene (feature) space is reduced with PCA to 50 PCs. A nearest neighbour graph and t-SNE are computed based on the PC space. Cell are clustered with leiden clustering based on the nearest neighbour graph. Graph abstraction is computed based on the leiden clustering.

In [15]:
if bool_recomp:
    adata_proc = adata_raw.copy()
    sc.pp.normalize_per_cell(adata_proc)
    adata_proc.raw = sc.pp.log1p(adata_proc, copy=True)
    sc.pp.pca(adata_proc, n_comps=50, random_state=0, svd_solver='arpack')
    sc.pp.neighbors(adata_proc, n_neighbors=100, knn=True, method='umap', n_pcs=50, random_state=0)
    sc.tl.tsne(adata_proc, n_jobs=3)
    sc.tl.umap(adata_proc)
    if bool_recluster==True:
        sc.tl.leiden(adata_proc, resolution=0.5)#, flavor='vtraag', random_state=0)
        pandas.DataFrame(adata_proc.obs).to_csv(
            path_or_buf =sc_settings_writedir+"obs_adata_proc.csv")
    else:
        obs = pandas.read_csv(sc_settings_writedir+'obs_adata_proc.csv')
        adata_proc.obs['leiden'] = pandas.Series(obs['leiden'].values, dtype='category')
    sc.write(sc_settings_writedir+'adata_proc.h5ad', adata_proc)
else:
    adata_proc = sc.read(sc_settings_writedir+'adata_proc.h5ad')
sc.tl.paga(adata_proc)

running PAGA
    finished: added
    'paga/connectivities', connectivities adjacency (adata.uns)
    'paga/connectivities_tree', connectivities subtree (adata.uns) (0:00:02)


Produce some summarizing plots that show the global characteristics of the data.

In [16]:
#Define a nice colour map for gene expression
colors2 = plt.cm.Reds(np.linspace(0, 1, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,20))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)

In [17]:
if bool_plot==True:
    sc.pl.tsne(adata_proc, color=['batch_type'], size=5, save="_all_age.pdf")
    sc.pl.tsne(adata_proc, color=['age_type'], size=5, save="_all_age.pdf")
    sc.pl.tsne(adata_proc, color=['leiden'], size=5, save="_all_leiden.pdf")
    sc.pl.tsne(adata_proc, color=['n_counts'], size=5, save="_all_n_counts.pdf")
    sc.pl.tsne(adata_proc, color=['Pdgfra'], size=5, save="_all_Pdgfra.pdf", color_map=mymap)
    sc.pl.tsne(adata_proc, color=['Igf2'], size=5, save="_all_Igf2.pdf", color_map=mymap)

In [18]:
if bool_plot==True:
    sc.pl.paga(adata_proc, save="_all.pdf")

Number of cells in each sample:

In [19]:
adata_proc.obs['batch_type'].value_counts()

PGF-2    5669
SCF-8    4936
SCF-2    3790
BAT-2    2317
PGF-8    2138
BAT-8    1889
Name: batch_type, dtype: int64

In [20]:
adata_proc.obs['leiden'].value_counts()

0     3789
1     2804
2     2405
3     2374
4     2236
5     1891
6     1770
7     1108
8     1034
9      653
10     432
11     114
12      82
13      47
Name: leiden, dtype: int64

# Define cell types

## Marker genes

### Define marker sets

Define surface marker sets for some of the expected cell types.

In [21]:
# Leukocyte markers:
leukocyte_markers = ['Ptprc']
tc_markers = ['Cd3d','Cd3e','Cd3g','Cd4'] 
nk_markers = ['Nkg7','Il2rb','Ncr1','Klrd1','Klrb1b','Klrb1f']
myeloid_markers = ['Cd79a','Itgax','Itgam','Fcgr3','S100a8', 'S100a9']
mp_markers = ['Adgre1','Lyz2']
dc_markers = ['Cd74','Anpep' , 'Cd33', 'Cd80', 'Cd83', 'Cd86'] 
bc_markers = ['Cd19']
adipocyte_markers = ['Pdgfra','Slc7a10','Pparg','Fermt2','Fbn1','Col4a1','Itgb1','Cd34','Cd24a','Dlk1','Slc7a10','Igf2']
megakaryocyte_markers = ['Ppbp']
go_adip_dev = ['Aacs','Acat1','Arid5b','Arrdc3','Atf2','Bbs4','Bdh1','Csf1','Dgat2','Dyrk1b','Ebf2','Amer1','Fto',
               'Id2','Lrp5','Nampt','Oxct1','Paxip1','Pik3ca','Ppard','Ppargc1a','Rorc','Sh3pxd2b','Slc25a25',
               'Sox8','Spg20','Tbl1xr1','Xbp1'] # ,'Lep'

### Plotting routines for marker gene sets:

In [22]:
def plot_violin_marker(adata, markers, save=None, use_raw=True):
    for i in range(len(markers) // 2 + len(markers) % 2):
        if save is not None:
            sc.pl.violin(
                adata, 
                groupby='leiden', 
                keys=markers[(2*i):np.min([2*(i+1), len(markers)])], 
                use_raw=use_raw, 
                rotation=90, size=5,
                save=save+"_"+str(i)+".pdf"
            )
        else:
            sc.pl.violin(
                adata, 
                groupby='leiden', 
                keys=markers[(2*i):np.min([2*(i+1), len(markers)])], 
                use_raw=use_raw, 
                rotation=90, size=5,
                save="dasdad"
            )
        
def plot_tsne_marker(adata, markers, size=5, save=None, use_raw=True):
    for i in range(len(markers) // 2 + len(markers) % 2):
        if save is not None:
            sc.pl.tsne(
                adata, 
                color=markers[(2*i):np.min([2*(i+1), len(markers)])], 
                size=size,
                use_raw=use_raw,
                color_map=mymap,
                save=save+"_"+str(i)+".pdf"
            )
        else:
            sc.pl.tsne(
                adata, 
                color=markers[(2*i):np.min([2*(i+1), len(markers)])], 
                size=size,
                use_raw=use_raw
            )

In [23]:
if bool_plot==True:
    plot_violin_marker(adata_proc, adipocyte_markers, save="_all_cells_adipocyte_markers")

### Leukocyte markers:

In [24]:
if bool_plot==True:
    plot_violin_marker(adata_proc, leukocyte_markers, save="_all_cells_leukocyte_markers")

Cluster 1,6,7,8,9 and potentially 0,10 express leukocyte marker Ptprc. These clusters are further validated by leukocyte specific markers below. The remaining clusters 2,3,4 are investigated with non-leukocyte marker sets.

### Megakaryocyte markers:

In [25]:
if bool_plot==True:   
    plot_violin_marker(adata_proc, megakaryocyte_markers, save="_all_cells_megakaryocyte_markers")

There do not seem to be many megakaryocytes in this data set.

### Preadipocyte markers:

In [26]:
if bool_plot==True:
    plot_violin_marker(adata_proc, adipocyte_markers, save="_all_cells_adipocyte_markers")

Cluster 2,3,4 express adipocyte markers.

### T-cell markers:

In [27]:
if bool_plot==True:
    plot_violin_marker(adata_proc, tc_markers, save="_all_cells_tc_markers")

Marker gene expression suggests that cluster 1 and 6 are T-cells, interestingly not Cd4+Cd8+ T-cells it seems as Cd4 expression is low.

### Natural killer cell markers:

In [28]:
if bool_plot==True:
    plot_violin_marker(adata_proc, nk_markers, save="_all_cells_nk_markers")

Cluster 1,6,7, have natural killer cell marker gene expression, cluster 1 also expresses Cd3 so it may contain yd-T-cells-?

### Myeloid cell markers:

In [29]:
if bool_plot==True:
    plot_violin_marker(adata_proc, myeloid_markers, save="_all_cells_myeloid_markers")

Cluster 5,8,9,10 express myeloid cell marker genes. Cluster 9 seems to have bimodal expression in S100a8 and S100a9 so it may need subclustering to subdivide cell types here.

### Macrophage markers:

In [30]:
if bool_plot==True:
    plot_violin_marker(adata_proc, mp_markers, save="_all_cells_mp_markers")

Cluster 5,7,8,9 express macrophage markers, in line with the myeloid cell marker gene expression.

### Dendritic cell markers:

In [31]:
if bool_plot==True:
    plot_violin_marker(adata_proc, dc_markers, save="_all_cells_dc_markers")

Clusters 0,5,7,8,9 express dendritic cell markers, in line with myeloid marker gene expression, cluster 0,7 could be a non-myeloid dendritic cell.

### B-cell markers:

In [32]:
if bool_plot==True:
    plot_violin_marker(adata_proc, bc_markers, save="_all_cells_bc_markers")

Cluster 0 may contain B-cells.

## Summary heatmap to characterize cell types

Select a few genes to summarize cell type assignments:

In [33]:
selected_leukocyte_markers = ['Ptprc']
selected_tc_markers = ['Cd3d','Cd3g']
selected_nk_markers = ['Nkg7','Klrd1']
selected_myeloid_markers = ['Fcgr3','S100a8']
selected_mp_markers = ['Adgre1','Lyz2']
selected_dc_markers = ['Cd74','Cd83'] 
selected_bc_markers = ['Cd19']
selected_adipocyte_markers = ['Pdgfra','Fbn1','Col4a1','Cd34','Cd24a','Dlk1','Slc7a10','Igf2'] 
selected_megakaryocyte_markers = ['Ppbp']

In [34]:
selected_cell_markers = selected_leukocyte_markers + \
selected_megakaryocyte_markers + \
selected_myeloid_markers + \
selected_mp_markers + \
selected_dc_markers + \
selected_bc_markers + \
selected_tc_markers + \
selected_nk_markers + \
selected_adipocyte_markers

Only keep markers that occur in data set.

In [35]:
if bool_plot==True:
    sc.pl.heatmap(
        adata=adata_proc, 
        var_names=selected_cell_markers, 
        groupby="leiden", 
        use_raw=True, 
        log=False, 
        dendrogram=False, 
        var_group_rotation=90, 
        show_gene_labels=True, 
        show=True, 
        save="_all_markers_celltypes.pdf"
    )

# Preadipocytes only

## Embedding and clustering

In [36]:
if bool_recomp==True:
    cell_ids_adip = np.asarray(adata_proc.obs_names)[[x in ['0','5','6'] 
                                                      for x in np.asarray(adata_proc.obs['leiden'].values)]]
    adata_adip = adata_raw[cell_ids_adip,:].copy()
    sc.pp.normalize_per_cell(adata_adip)
    adata_adip.raw = adata_adip.copy()
    sc.pp.log1p(adata_adip)
    sc.pp.pca(adata_adip, n_comps=50, random_state=0, svd_solver='arpack')
    sc.pp.neighbors(adata_adip, n_neighbors=100, knn=True, method='umap', n_pcs=50, random_state=0)
    sc.tl.tsne(adata_adip, n_jobs=3)
    if bool_recluster==True:
        sc.tl.leiden(adata_adip, resolution=0.5)#, flavor='vtraag', random_state=0)
        pandas.DataFrame(adata_adip.obs).to_csv(
            path_or_buf=sc_settings_writedir+"obs_adata_adip.csv")
    else:
        obs = pandas.read_csv(sc_settings_writedir+'obs_adata_adip.csv')
        adata_adip.obs['leiden'] = pandas.Series(obs['leiden'].values, dtype='category')
    sc.write(sc_settings_writedir+'adata_adip.h5ad', adata_adip)
else:
    adata_adip = sc.read(sc_settings_writedir+'adata_adip.h5ad')
sc.tl.paga(adata_adip)

running PAGA
    finished: added
    'paga/connectivities', connectivities adjacency (adata.uns)
    'paga/connectivities_tree', connectivities subtree (adata.uns) (0:00:00)


In [37]:
if bool_plot==True:
    sc.pl.tsne(adata_adip, color=['batch_type'], size=20, save="_preadip_batch.pdf")
    sc.pl.tsne(adata_adip, color=['leiden'], size=20, save="_preadip_leiden.pdf")
    sc.pl.tsne(adata_adip, color=['n_counts'], size=20, save="_preadip_n_counts.pdf")

Fairly equal count depth across preadipocytes. Potential batch effect or age effect, indistinguishable in this scenario.

In [38]:
if bool_plot==True:
    sc.pl.paga(adata_adip, save="_preadip.pdf")

It seems that Slc7a10+ cells are in clusters 0,2,4, which directly correspond to the samples of the young mice with a few exceptions. 

# Define cell types in preadipocytes

In [39]:
if bool_plot==True:
    plot_violin_marker(adata_adip, adipocyte_markers, save="_preadipocytes_cells_adipocyte_markers")

### Leukocyte markers:

In [40]:
if bool_plot==True:
    plot_violin_marker(adata_adip, leukocyte_markers, save="_preadipocytes_cells_leukocyte_markers")

Cluster 1,6,7,8,9 and potentially 0,10 express leukocyte marker Ptprc. These clusters are further validated by leukocyte specific markers below. The remaining clusters 2,3,4 are investigated with non-leukocyte marker sets.

### Megakaryocyte markers:

In [41]:
if bool_plot==True:   
    plot_violin_marker(adata_adip, megakaryocyte_markers, save="_preadipocytes_cells_megakaryocyte_markers")

There do not seem to be many megakaryocytes in this data set.

### Preadipocyte markers:

In [42]:
if bool_plot==True:
    plot_violin_marker(adata_adip, adipocyte_markers, save="_preadipocytes_cells_adipocyte_markers")

Cluster 2,3,4 express adipocyte markers.

### T-cell markers:

In [43]:
if bool_plot==True:
    plot_violin_marker(adata_adip, tc_markers, save="_preadipocytes_cells_tc_markers")

Marker gene expression suggests that cluster 1 and 6 are T-cells, interestingly not Cd4+Cd8+ T-cells it seems as Cd4 expression is low.

### Natural killer cell markers:

In [44]:
if bool_plot==True:
    plot_violin_marker(adata_adip, nk_markers, save="_preadipocytes_cells_nk_markers")

Cluster 1,6,7, have natural killer cell marker gene expression, cluster 1 also expresses Cd3 so it may contain yd-T-cells-?

### Myeloid cell markers:

In [45]:
if bool_plot==True:
    plot_violin_marker(adata_adip, myeloid_markers, save="_preadipocytes_cells_myeloid_markers")

Cluster 5,8,9,10 express myeloid cell marker genes. Cluster 9 seems to have bimodal expression in S100a8 and S100a9 so it may need subclustering to subdivide cell types here.

### Macrophage markers:

In [46]:
if bool_plot==True:
    plot_violin_marker(adata_adip, mp_markers, save="_preadipocytes_cells_mp_markers")

Cluster 5,7,8,9 express macrophage markers, in line with the myeloid cell marker gene expression.

### Dendritic cell markers:

In [47]:
if bool_plot==True:
    plot_violin_marker(adata_adip, dc_markers, save="_preadipocytes_cells_dc_markers")

Clusters 0,5,7,8,9 express dendritic cell markers, in line with myeloid marker gene expression, cluster 0,7 could be a non-myeloid dendritic cell.

### B-cell markers:

In [48]:
if bool_plot==True:
    plot_violin_marker(adata_adip, bc_markers, save="_preadipocytes_cells_bc_markers")

Cluster 0 may contain B-cells.

## Marker gene sets

In [49]:
if bool_plot==True:
    plot_tsne_marker(adata_adip, adipocyte_markers, size=20, save="_preadipocytes_cells_adipocyte_markers", use_raw=False)

# Non-preadipocytes only

## Embedding and clustering

In [50]:
if bool_recomp==True:
    cell_ids_imun = np.asarray(adata_proc.obs_names)[[x in ['1','2','3','4','7','8','9','10','11','12','13'] 
                                                      for x in np.asarray(adata_proc.obs['leiden'].values)]]
    adata_imun = adata_raw[cell_ids_imun,:].copy()
    sc.pp.normalize_per_cell(adata_imun)
    adata_imun.raw = adata_imun.copy()
    sc.pp.log1p(adata_imun)
    sc.pp.pca(adata_imun, n_comps=50, random_state=0, svd_solver='arpack')
    sc.pp.neighbors(adata_imun, n_neighbors=100, knn=True, method='umap', n_pcs=50, random_state=0)
    sc.tl.tsne(adata_imun, n_jobs=3)
    if bool_recluster==True:
        sc.tl.leiden(adata_imun, resolution=0.5)#, flavor='vtraag', random_state=0)
        pandas.DataFrame(adata_imun.obs).to_csv(
            path_or_buf=sc_settings_writedir+"obs_adata_imun.csv")
    else:
        obs = pandas.read_csv(sc_settings_writedir+'obs_adata_imun.csv')
        adata_imun.obs['leiden'] = pandas.Series(obs['leiden'].values, dtype='category')
    sc.write(sc_settings_writedir+'adata_imun.h5ad', adata_imun)
else:
    adata_imun = sc.read(sc_settings_writedir+'adata_imun.h5ad')
sc.tl.paga(adata_imun)

running PAGA
    finished: added
    'paga/connectivities', connectivities adjacency (adata.uns)
    'paga/connectivities_tree', connectivities subtree (adata.uns) (0:00:01)


In [51]:
if bool_plot==True:
    sc.pl.tsne(adata_imun, color=['batch_type'], size=20, save="_nonpreadipo_batch.pdf")
    sc.pl.tsne(adata_imun, color=['leiden'], size=20, save="_nonpreadip_leiden.pdf")
    sc.pl.tsne(adata_imun, color=['n_counts'], size=20, save="_nonpreadip_n_counts.pdf")

Fairly equal count depth across preadipocytes. Potential batch effect or age effect, indistinguishable in this scenario.

In [52]:
if bool_plot==True:
    sc.pl.paga(adata_imun, save="_preadip.pdf")

In [53]:
if bool_plot==True:
    plot_violin_marker(adata_imun, ['Igf2'], save="_preadipocytes_cells_bc_markers", use_raw=False)

# Preadipocytes only by SCF

## Embedding and clustering

### SCF-2

In [54]:
if bool_recomp==True:
    cell_ids_adip_young = np.asarray(adata_proc.obs_names)[
        np.logical_and(np.asarray([x in ['0','5','6']
                                   for x in np.asarray(adata_proc.obs['leiden'].values)]),
                       np.asarray([x=='SCF-2'
                                   for x in np.asarray(adata_proc.obs['batch_type'].values)]))]
    adata_adip_young = adata_raw[cell_ids_adip_young,:].copy()
    #sc.pp.filter_cells(adata_adip_young, min_counts=500)
    sc.pp.normalize_per_cell(adata_adip_young)
    adata_adip_young.raw = adata_adip_young.copy()
    sc.pp.log1p(adata_adip_young)
    sc.pp.pca(adata_adip_young, n_comps=50, random_state=0, svd_solver='arpack')
    sc.pp.neighbors(adata_adip_young, n_neighbors=100, knn=True, method='umap', n_pcs=50, random_state=0)
    sc.tl.tsne(adata_adip_young, n_jobs=3)
    if bool_recluster==True:
        sc.tl.leiden(adata_adip_young, resolution=0.5)#, flavor='vtraag', random_state=0)
        pandas.DataFrame(adata_adip_young.obs).to_csv(
            path_or_buf =sc_settings_writedir+"obs_adata_adip_young.csv")
    else:
        obs = pandas.read_csv(sc_settings_writedir+'obs_adata_adip_young.csv')
        adata_adip_young.obs['leiden'] = pandas.Series(obs['leiden'].values, dtype='category')
    sc.write(sc_settings_writedir+'adata_adip_young.h5ad', adata_adip_young)
else:
    adata_adip_young = sc.read(sc_settings_writedir+'adata_adip_young.h5ad')
sc.tl.paga(adata_adip_young)

running PAGA
    finished: added
    'paga/connectivities', connectivities adjacency (adata.uns)
    'paga/connectivities_tree', connectivities subtree (adata.uns) (0:00:00)


In [55]:
if bool_plot==True:
    sc.pl.tsne(adata_adip_young, color=['leiden'], size=20, save="_preadipYoung_leiden.pdf")
    sc.pl.tsne(adata_adip_young, color=['n_counts'], size=20, save="_preadipYoung_n_counts.pdf")
    sc.pl.paga(adata_adip_young, save="_preadipYoung.pdf")

Number of preadipocytes observed in young mouse with scRNAseq:

In [56]:
print(adata_adip_young.X.shape[0])

1648


### SCF-2 - coarse clustering

In [57]:
adata_adip_young_lowres = adata_adip_young.copy()
if True:
    if True:
        sc.tl.leiden(adata_adip_young_lowres, resolution=0.2)#, flavor='vtraag', random_state=0)
        pandas.DataFrame(adata_adip_young_lowres.obs).to_csv(
            path_or_buf =sc_settings_writedir+"obs_adata_adip_young_lowres.csv")
    else:
        obs = pandas.read_csv(sc_settings_writedir+'obs_adata_adip_young_lowres.csv')
        adata_adip_young_lowres.obs['leiden'] = pandas.Series(obs['leiden'].values, dtype='category')

running Leiden clustering
    finished: found 2 clusters and added
    'leiden', the cluster labels (adata.obs, categorical) (0:00:00)


In [58]:
if bool_plot==True:
    sc.pl.tsne(adata_adip_young_lowres, color=['leiden'], size=20, save="_preadipYoung_leiden_lowres.pdf")

In [59]:
adata_adip_young_lowres.obs['leiden'].value_counts()

0    1117
1     531
Name: leiden, dtype: int64

### SCF-8

In [60]:
if bool_recomp==True:
    cell_ids_adip_old = np.asarray(adata_proc.obs_names)[
        np.logical_and(np.asarray([x in ['0','5','6']
                                   for x in np.asarray(adata_proc.obs['leiden'].values)]),
                       np.asarray([x=='SCF-8'
                                   for x in np.asarray(adata_proc.obs['batch_type'].values)]))]
    adata_adip_old = adata_raw[cell_ids_adip_old,:].copy()
    sc.pp.filter_cells(adata_adip_old, min_counts=500)
    sc.pp.normalize_per_cell(adata_adip_old)
    adata_adip_old.raw = adata_adip_old.copy()
    sc.pp.log1p(adata_adip_old)
    sc.pp.pca(adata_adip_old, n_comps=50, random_state=0, svd_solver='arpack')
    sc.pp.neighbors(adata_adip_old, n_neighbors=50, knn=True, method='umap', n_pcs=50, random_state=0)
    sc.tl.tsne(adata_adip_old, n_jobs=3)
    if bool_recluster==True:
        sc.tl.leiden(adata_adip_old, resolution=0.5)#, flavor='vtraag', random_state=0)
        pandas.DataFrame(adata_adip_old.obs).to_csv(
            path_or_buf =sc_settings_writedir+"obs_adata_adip_old.csv")
    else:
        obs = pandas.read_csv(sc_settings_writedir+'obs_adata_adip_old.csv')
        adata_adip_old.obs['leiden'] = pandas.Series(obs['leiden'].values, dtype='category')
    sc.write(sc_settings_writedir+'adata_adip_old.h5ad', adata_adip_old)
else:
    adata_adip_old = sc.read(sc_settings_writedir+'adata_adip_old.h5ad')
sc.tl.paga(adata_adip_old)

running PAGA
    finished: added
    'paga/connectivities', connectivities adjacency (adata.uns)
    'paga/connectivities_tree', connectivities subtree (adata.uns) (0:00:00)


In [61]:
if bool_plot==True:
    sc.pl.tsne(adata_adip_old, color=['leiden'], size=20, save="_preadipOld_leiden.pdf")
    sc.pl.tsne(adata_adip_old, color=['n_counts'], size=20, save="_preadipOld_n_counts.pdf")
    sc.pl.paga(adata_adip_old, save="_preadipOld.pdf")

Number of preadipocytes observed in old mouse with scRNAseq:

In [62]:
print(adata_adip_old.X.shape[0])

1409


## Marker gene sets

### SCF-2

In [63]:
adipocyte_markers=['Pdgfra','Slc7a10','Pparg','Fermt2','Fbn1','Col4a1','Itgb1','Cd34','Cd24a','Dlk1','Srr','Fabp4','Dpp4','Igf2']

In [64]:
if bool_plot==True:
    plot_violin_marker(adata_adip_young, adipocyte_markers, save="_preadipYoung_adipocyte_markers", use_raw=False)

In [65]:
if bool_plot==True:
    plot_violin_marker(adata_adip_young, go_adip_dev, save="_preadipYoung_GO_adipocyte_dev_markers", use_raw=False)

In [66]:
if bool_plot==True:
    plot_tsne_marker(adata_adip_young, adipocyte_markers, size=20, save="_preadipYoung_cells_adipocyte_markers", use_raw=False)

In [67]:
if bool_plot==True:
    plot_tsne_marker(adata_adip_young, go_adip_dev, size=20, save="_preadipYoung_cells_GO_adipocyte_dev_markers", use_raw=False)

In [68]:
if bool_plot==True:
    sc.pl.heatmap(
        adata=adata_adip_young, 
        var_names=adipocyte_markers, 
        groupby="leiden", 
        use_raw=True, 
        log=True, 
        cmap="viridis",
        vmin=-1,
        vmax=4,
        dendrogram=False, 
        var_group_rotation=90, 
        show_gene_labels=True, 
        show=True, 
        save="_preadipYoung_markers_preadipocytes.pdf"
    )

In [69]:

if bool_plot==True:
    sc.pl.heatmap(
        adata=adata_adip_young, 
        var_names=go_adip_dev, 
        groupby="leiden", 
        use_raw=True, 
        log=True, 
        cmap="viridis",
        vmin=-1,
        vmax=2,
        dendrogram=False, 
        var_group_rotation=90, 
        show_gene_labels=True, 
        show=True, 
        save="_preadipYoung_markers_GO_adipocyte_dev.pdf"
    )

In [70]:
[{'mean expression of Igf2 in leiden group '+x:
  np.mean(adata_adip_young[adata_adip_young.obs['leiden'].values==x,:][:,'Igf2'].X)} 
 for x in adata_adip_young.obs['leiden'].cat.categories.values]

[{'mean expression of Igf2 in leiden group 0': 0.7055549},
 {'mean expression of Igf2 in leiden group 1': 0.4558921},
 {'mean expression of Igf2 in leiden group 2': 0.7972599}]

This seems to be a continuum of cell types with two opposing gradients of gene expression: Pdgfra and Slc7a10. This could indicate a developmental lineage.

### SCF-2 - coarse clustering

In [71]:
if bool_plot==True:
    plot_violin_marker(adata_adip_young_lowres, adipocyte_markers, save="_preadipYoung_lowres_adipocyte_markers")

In [72]:
if bool_plot==True:
    plot_violin_marker(adata_adip_young_lowres, go_adip_dev, save="_preadipYoung_lowres_GO_adipocyte_dev_markers")

In [73]:
if bool_plot==True:
    sc.pl.heatmap(
        adata=adata_adip_young_lowres, 
        var_names=adipocyte_markers, 
        groupby="leiden", 
        use_raw=True, 
        log=True, 
        dendrogram=False, 
        var_group_rotation=90, 
        show_gene_labels=True, 
        show=True, 
        save="_preadipYoung_lowres_markers_preadipocytes.pdf"
    )

In [74]:
if bool_plot==True:
    sc.pl.heatmap(
        adata=adata_adip_young_lowres, 
        var_names=go_adip_dev, 
        groupby="leiden", 
        use_raw=True, 
        log=True, 
        dendrogram=False, 
        var_group_rotation=90, 
        show_gene_labels=True, 
        show=True, 
        save="_preadipYoung_lowres_markers_GO_adipocyte_dev.pdf"
    )

### SCF-8

In [75]:
if bool_plot==True:
    plot_violin_marker(adata_adip_old, adipocyte_markers, save="_preadipOld_adipocyte_markers")

In [76]:
if bool_plot==True:
    plot_violin_marker(adata_adip_old, go_adip_dev, save="_preadipOld_GO_adipocyte_dev_markers")

In [77]:
if bool_plot==True:
    plot_tsne_marker(adata_adip_old, adipocyte_markers, size=10, save="_preadipOld_adipocyte_markers")

In [78]:
if bool_plot==True:
    plot_tsne_marker(adata_adip_old, go_adip_dev, size=10, save="_preadipOld_GO_adipocyte_dev_markers")

In [79]:
if bool_plot==True:
    sc.pl.heatmap(
        adata=adata_adip_old, 
        var_names=adipocyte_markers, 
        groupby="leiden", 
        use_raw=True, 
        log=True, 
        dendrogram=False, 
        var_group_rotation=90, 
        show_gene_labels=True, 
        show=True, 
        save="_preadipOld_markers_preadipocytes.pdf"
    )

In [80]:
if bool_plot==True:
    sc.pl.heatmap(
        adata=adata_adip_old, 
        var_names=go_adip_dev, 
        groupby="leiden", 
        use_raw=True, 
        log=True, 
        dendrogram=False, 
        var_group_rotation=90, 
        show_gene_labels=True, 
        show=True, 
        save="_preadipOld_markers_GO_adipocyte_dev.pdf"
    )

This looks like there are two clusters, one consisting of leiden group 0,4 (Pdgfra-Slc7a10-) and one of group 1,2,3,5 (Pdgfra+Slc7a10-).

# Differential expression analysis

## By BAT

In [81]:
cell_ids_bat = np.asarray(adata_proc.obs_names)[
        np.logical_and(np.asarray([x in ['3','4','5','7','10']
                                   for x in np.asarray(adata_proc.obs['leiden'].values)]),
                       np.asarray([x=='BAT'
                                   for x in np.asarray(adata_proc.obs['tissue_type'].values)]))]
adata_bat = adata_raw[cell_ids_bat,:].copy()
if bool_plot==True:
    adata_bat.obs

In [82]:
dets_bat = de.test.t_test(
    data=adata_bat, 
    sample_description=adata_bat.obs, 
    grouping="age_type", 
    is_logged=False
)

The differentially expressed genes with some fold-change and mean expression thresholding are:

In [83]:
dets_bat_summary = dets_bat.summary(
    mean_thres=np.log(0.01), qval_thres=0.05, 
    fc_lower_thres=0.5, fc_upper_thres=2
)
dets_bat_summary.to_csv(path_or_buf=dir_tables+"DE_preadip_by_bat.csv", sep="\t")
if bool_plot==True:
    dets_bat_summary

In [84]:
if bool_plot==True:
    dets_bat.plot_volcano(
        alpha=0.01, min_fc=1.5, size=15, log10_p_threshold=-50, log2_fc_threshold=7,
        highlight_ids=['Igf2'], highlight_size=30, highlight_col="red",
        save=sc_settings_figdir+"preadip_DE_bat", suffix="_volcano.pdf"
    )

In [85]:
if bool_plot==True:
    dets_bat.plot_ma(
        alpha=0.01, log2_fc_threshold=6,
        highlight_ids=['Igf2'], highlight_size=30, highlight_col="red",
        save=sc_settings_figdir+"preadip_DE_bat", suffix="_ma_plot.pdf"
    )

Count DE genes:

In [86]:
np.sum(np.array([x < 0.01 if x is not np.nan else False for x in dets_bat.qval]))

3583

## By SCF

In [87]:
cell_ids_scf = np.asarray(adata_proc.obs_names)[
        np.logical_and(np.asarray([x in ['3','4','5','7','10']
                                   for x in np.asarray(adata_proc.obs['leiden'].values)]),
                       np.asarray([x=='SCF'
                                   for x in np.asarray(adata_proc.obs['tissue_type'].values)]))]
adata_scf = adata_raw[cell_ids_scf,:].copy()

In [88]:
dets_scf = de.test.t_test(
    data=adata_scf, 
    sample_description=adata_scf.obs, 
    grouping="age_type", 
    is_logged=False
)

The differentially expressed genes with some fold-change and mean expression thresholding are:

In [89]:
dets_scf_summary = dets_scf.summary(
    mean_thres=np.log(0.01), qval_thres=0.05, 
    fc_lower_thres=0.5, fc_upper_thres=2
)
dets_scf_summary.to_csv(path_or_buf=dir_tables+"DE_preadip_by_scf.csv", sep="\t")
if bool_plot==True:
    dets_scf_summary

In [90]:
if bool_plot==True:
    dets_scf.plot_volcano(
        alpha=0.01, min_fc=1.5, size=15, log10_p_threshold=-50, log2_fc_threshold=7,
        highlight_ids=['Igf2'], highlight_size=30, highlight_col="red",
        save=sc_settings_figdir+"preadip_DE_scf", suffix="_volcano.pdf"
    )

In [91]:
if bool_plot==True:
    dets_scf.plot_ma(
        alpha=0.01, log2_fc_threshold=6,
        highlight_ids=['Igf2'], highlight_size=30, highlight_col="red",
        save=sc_settings_figdir+"preadip_DE_scf", suffix="_ma_plot.pdf"
    )

Count DE genes:

In [92]:
np.sum(np.array([x < 0.01 if x is not np.nan else False for x in dets_scf.qval]))

2441

## By PGF

In [93]:
cell_ids_pgf = np.asarray(adata_proc.obs_names)[
        np.logical_and(np.asarray([x in ['3','4','5','7','10']
                                   for x in np.asarray(adata_proc.obs['leiden'].values)]),
                       np.asarray([x=='PGF'
                                   for x in np.asarray(adata_proc.obs['tissue_type'].values)]))]
adata_pgf = adata_raw[cell_ids_pgf,:].copy()
if bool_plot==True:
    adata_pgf.obs

In [94]:
dets_pgf = de.test.t_test(
    data=adata_pgf, 
    sample_description=adata_pgf.obs, 
    grouping="age_type", 
    is_logged=False
)

The differentially expressed genes with some fold-change and mean expression thresholding are:

In [95]:
dets_pgf_summary = dets_pgf.summary(
    mean_thres=np.log(0.01), qval_thres=0.05, 
    fc_lower_thres=0.5, fc_upper_thres=2
)
dets_pgf_summary.to_csv(path_or_buf=dir_tables+"DE_preadip_by_pgf.csv", sep="\t")
if bool_plot==True:
    dets_pgf_summary

In [96]:
if bool_plot==True:
    dets_pgf.plot_volcano(
        alpha=0.01, min_fc=1.5, size=15, log10_p_threshold=-50, log2_fc_threshold=7,
        highlight_ids=['Igf2'], highlight_size=30, highlight_col="red",
        save=sc_settings_figdir+"preadip_DE_pgf", suffix="_volcano.pdf"
    )

In [97]:
if bool_plot==True:
    dets_pgf.plot_ma(
        alpha=0.01, log2_fc_threshold=6,
        highlight_ids=['Igf2'], highlight_size=30, highlight_col="red",
        save=sc_settings_figdir+"preadip_DE_pgf", suffix="_ma_plot.pdf"
    )

Count DE genes:

In [98]:
np.sum(np.array([x < 0.01 if x is not np.nan else False for x in dets_pgf.qval]))

13274

## By leiden group

### Coarse clustering

#### Test

Here, we perform a differential expression test across the coarse-grained louvain clustering which yielded two groups.

In [99]:
dets_young_lowres_louvain = de.test.t_test(
    data=adata_adip_young_lowres.raw, 
    sample_description=adata_adip_young_lowres.obs, 
    grouping="leiden", 
    is_logged=False
)

The differentially expressed genes with some fold-change and mean expression thresholding are:

In [100]:
dets_young_lowres_louvain_summary = dets_young_lowres_louvain.summary(
    mean_thres=np.log(0.01), qval_thres=0.05, 
    fc_lower_thres=0.5, fc_upper_thres=2
)
dets_young_lowres_louvain_summary.to_csv(path_or_buf=dir_tables+"DE_preadipYoung_by_louvain_lowres.csv", sep="\t")
if bool_plot==True:
    dets_young_lowres_louvain_summary

In [101]:
if bool_plot==True:
    dets_young_lowres_louvain.plot_volcano(
        alpha=0.01, min_fc=1.5, size=15, log10_p_threshold=-15, log2_fc_threshold=5,
        highlight_ids=['Igf2'], highlight_size=30, highlight_col="red",
        save=sc_settings_figdir+"preadipYoung_DE_louvain_lowres", suffix="_volcano.pdf"
    )

In [102]:
if bool_plot==True:
    dets_young_lowres_louvain.plot_ma(
        alpha=0.01, log2_fc_threshold=5,
        highlight_ids=['Igf2'], highlight_size=30, highlight_col="red",
        save=sc_settings_figdir+"preadipYoung_DE_louvain_lowres", suffix="_ma_plot.pdf"
    )

Count DE genes:

In [103]:
np.sum(np.array([x < 0.01 if x is not np.nan else False for x in dets_young_lowres_louvain.qval]))

2561

Save DE genes to file for enrichment:

In [104]:
dets_young_lowres_louvain.summary(qval_thres=0.01)["gene"].to_csv(dir_out+"enrichment/de_genes_young_preadip.csv", 
                                                                  index=False
)

## GO and KEGG enrichment analysis

In [105]:
glist=dets_young_lowres_louvain_summary.loc[dets_young_lowres_louvain_summary['pval'] < 0.05]['gene'].squeeze().str.strip().tolist()

In [106]:
enr_go = gseapy.enrichr(gene_list=glist,
                     organism='Mouse',
                     gene_sets='GO_Biological_Process_2018',
                     description='pathway',
                     cutoff = 0.5
)
enr_go.results[enr_go.results['Adjusted P-value']<0.05].to_csv(path_or_buf=dir_tables+"GO_enrichment_young_lowres_louvain.csv", sep="\t")
if bool_plot==True:
    enr_go.results[enr_go.results['Adjusted P-value']<0.05]

In [107]:
if bool_plot==True:
    gseapy.barplot(enr_go.res2d,
                   title='GO Enrichment'
    )

In [108]:
enr_kegg = gseapy.enrichr(gene_list=glist,
                        organism='Mouse',
                        gene_sets='KEGG_2019_Mouse',
                        description='pathway',
                        cutoff = 0.5
)
enr_kegg.results[enr_kegg.results['Adjusted P-value']<0.05].to_csv(path_or_buf=dir_tables+"KEGG_enrichment_young_lowres_louvain.csv",
                                                               sep="\t"
)
if bool_plot==True:
    enr_kegg.results[enr_kegg.results['Adjusted P-value']<0.05]

In [109]:
if bool_plot==True:
    gseapy.barplot(enr_kegg.res2d,
                   title='KEGG enrichment'
    )

### Gene comparison

In [110]:
group1 = ['Igf2','Pparg','Adipoq','Fabp4','Cd34','Fgf21','Stat3','Fn1'] # ,'Col4a','Col6a'
group2 = ['Cd55','Cd34','Fbn1','Anxa3','Mfap5','Timp2','Sparc','Cyr61','Klf2','Fabp4','Lpl','Wt1','Rbp1','Igf1',
          'Col4a1','Col4a2','Col6a2','Fstl1','Icam1']# ,'Cd142'

In [111]:
if bool_plot==True:
    sc.pl.stacked_violin(
                adata=adata_adip_young, 
                groupby='leiden',
                var_names=group1, 
                save='_group1_scf_young', 
                figsize=(10,15),
                swap_axes=True)
    sc.pl.dotplot(
                adata=adata_adip_young,
                groupby='leiden',
                var_names=group1, 
                save='_group1_scf_young')

In [112]:
if bool_plot==True:
    sc.pl.stacked_violin(
                adata=adata_adip, 
                groupby='leiden',
                var_names=group1, 
                save='_group1_all_preadip', 
                figsize=(10,15),
                swap_axes=True)
    sc.pl.dotplot(
                adata=adata_adip,
                groupby='leiden',
                var_names=group1, 
                save='_group1_all_preadip')

In [113]:
if bool_plot==True:
    sc.pl.stacked_violin(adata=adata_adip, 
                         groupby='leiden',
                         var_names=group2, 
                         save='_group2_all_preadip', 
                         figsize=(10,15),
                         swap_axes=True
    )
    sc.pl.dotplot(adata=adata_adip,
                  groupby='leiden',
                  var_names=group2, 
                  save='_group2_all_preadip'
    )

#### Heatmaps

Select all differentially expressed genes at a corrected p-value threshold of 0.01 and a minimal or maximal log2 fold change of 2 or 0.5 and a minimal mean expression of 0.5.

In [114]:
dets_young_lowres_louvain_summary_forheatmap = dets_young_lowres_louvain.summary(
    mean_thres=0.5, qval_thres=0.01, 
    fc_lower_thres=0.5, fc_upper_thres=2
)

In [115]:
all_de_genes_young_coarse = dets_young_lowres_louvain_summary_forheatmap['gene'].values[
    np.argsort(dets_young_lowres_louvain_summary_forheatmap['log2fc'].values)
]

In [116]:
print(len(all_de_genes_young_coarse))

253


In [117]:
if bool_plot==True:
    sc.pl.heatmap(
        adata=adata_adip_young_lowres, 
        var_names=all_de_genes_young_coarse, 
        groupby="leiden", 
        use_raw=False, 
        log=False, 
        cmap="viridis",
        vmin=-1,
        vmax=5,
        dendrogram=False, 
        var_group_rotation=90, 
        show_gene_labels=True, 
        show=True, 
        save="_preadipYoung_lowres_all_de_genes.pdf"
    )